## Imports

In [14]:
import torch
import pandas as pd

from data_generation import (
    generate_objects,
    build_binary_relation_dataset,
    build_inbetween_dataset,
    left_of,
    right_of,
    close_to,
    above,
    below
)

from predicates import LeftOf, RightOf, CloseTo, Above, Below, InBetween
from axioms import create_knowledge_base
from trainer import train_ltn
from metrics import compute_metrics

## Configuração inicial

In [15]:
# Configuração para experimento completo
NUM_RUNS = 5
EPOCHS = 500

all_results = []

## Loop das 5 execuções

## Teste rápido com 1 execução

In [16]:
import ltn

# Teste com 1 execução e poucos epochs
print("Testando 1 execução...")

objects = generate_objects(n=10, seed=42)  # Poucos objetos para teste
print(f"Objetos: {objects.shape}")

# Treinar com poucos epochs
sat = train_ltn(create_knowledge_base, objects, epochs=20, verbose=True)
print(f"satAgg final = {sat:.4f}")

# Criar variáveis LTN para avaliação
x_var = ltn.Variable("x", objects)
y_var = ltn.Variable("y", objects)

# Testar avaliação LeftOf
X, y_true = build_binary_relation_dataset(objects, left_of)
print(f"Dataset LeftOf: X={X.shape}, y_true={y_true.shape}")

with torch.no_grad():
    # Pegar predições LTN
    ltn_pred = LeftOf(x_var, y_var).value  # Shape: [10, 10]
    
    # Extrair apenas pares onde i != j (mesma lógica do dataset)
    n = len(objects)
    y_pred = []
    for i in range(n):
        for j in range(n):
            if i != j:
                y_pred.append(ltn_pred[i, j])
    
    y_pred = torch.stack(y_pred)
    print(f"y_pred após filtrar: {y_pred.shape}")
    
acc, prec, rec, f1 = compute_metrics(y_true, y_pred)
print(f"LeftOf - Acc: {acc:.4f}, Prec: {prec:.4f}, Rec: {rec:.4f}, F1: {f1:.4f}")

print("✅ Teste OK - Pipeline funcionando!")

Testando 1 execução...
Objetos: torch.Size([10, 11])
Epoch 0000 | satAgg = 0.9998
Epoch 0005 | satAgg = 0.9998
Epoch 0010 | satAgg = 0.9998
Epoch 0015 | satAgg = 0.9998
satAgg final = 0.9998
Dataset LeftOf: X=torch.Size([90, 22]), y_true=torch.Size([90])
y_pred após filtrar: torch.Size([90])
LeftOf - Acc: 0.5000, Prec: 0.0000, Rec: 0.0000, F1: 0.0000
✅ Teste OK - Pipeline funcionando!


In [17]:
import ltn

def extract_predictions(ltn_pred, n_objects):
    """Extrai predições excluindo pares i=j (mesma lógica do dataset)"""
    y_pred = []
    for i in range(n_objects):
        for j in range(n_objects):
            if i != j:
                y_pred.append(ltn_pred[i, j])
    return torch.stack(y_pred)

def extract_ternary_predictions(ltn_pred, n_objects):
    """Extrai predições ternárias excluindo casos onde algum índice se repete"""
    y_pred = []
    for i in range(n_objects):
        for j in range(n_objects):
            for k in range(n_objects):
                if i != j and j != k and i != k:
                    y_pred.append(ltn_pred[i, j, k])
    return torch.stack(y_pred)

for run in range(NUM_RUNS):
    print(f"\n==============================")
    print(f" Execução {run + 1}")
    print(f"==============================")

    # 1. Gerar objetos
    objects = generate_objects(n=25, seed=run)
    n = len(objects)

    # 2. Treinar LTN (passando função que cria KB)
    sat = train_ltn(create_knowledge_base, objects, epochs=EPOCHS, verbose=False)
    print(f"satAgg = {sat:.4f}")

    # Criar variáveis LTN para avaliação
    x_var = ltn.Variable("x", objects)
    y_var = ltn.Variable("y", objects)
    z_var = ltn.Variable("z", objects)

    # ------------------------------
    # LEFT OF
    # ------------------------------
    X, y_true = build_binary_relation_dataset(objects, left_of)
    with torch.no_grad():
        ltn_pred = LeftOf(x_var, y_var).value
        y_pred = extract_predictions(ltn_pred, n)
    acc, prec, rec, f1 = compute_metrics(y_true, y_pred)

    all_results.append({
        "Run": run + 1,
        "Predicate": "LeftOf",
        "satAgg": sat,
        "Accuracy": acc,
        "Precision": prec,
        "Recall": rec,
        "F1": f1
    })

    # ------------------------------
    # RIGHT OF
    # ------------------------------
    X, y_true = build_binary_relation_dataset(objects, right_of)
    with torch.no_grad():
        ltn_pred = RightOf(x_var, y_var).value
        y_pred = extract_predictions(ltn_pred, n)
    acc, prec, rec, f1 = compute_metrics(y_true, y_pred)

    all_results.append({
        "Run": run + 1,
        "Predicate": "RightOf",
        "satAgg": sat,
        "Accuracy": acc,
        "Precision": prec,
        "Recall": rec,
        "F1": f1
    })

    # ------------------------------
    # CLOSE TO
    # ------------------------------
    X, y_true = build_binary_relation_dataset(objects, close_to)
    with torch.no_grad():
        ltn_pred = CloseTo(x_var, y_var).value
        y_pred = extract_predictions(ltn_pred, n)
    acc, prec, rec, f1 = compute_metrics(y_true, y_pred)

    all_results.append({
        "Run": run + 1,
        "Predicate": "CloseTo",
        "satAgg": sat,
        "Accuracy": acc,
        "Precision": prec,
        "Recall": rec,
        "F1": f1
    })

    # ------------------------------
    # ABOVE
    # ------------------------------
    X, y_true = build_binary_relation_dataset(objects, above)
    with torch.no_grad():
        ltn_pred = Above(x_var, y_var).value
        y_pred = extract_predictions(ltn_pred, n)
    acc, prec, rec, f1 = compute_metrics(y_true, y_pred)

    all_results.append({
        "Run": run + 1,
        "Predicate": "Above",
        "satAgg": sat,
        "Accuracy": acc,
        "Precision": prec,
        "Recall": rec,
        "F1": f1
    })

    # ------------------------------
    # BELOW
    # ------------------------------
    X, y_true = build_binary_relation_dataset(objects, below)
    with torch.no_grad():
        ltn_pred = Below(x_var, y_var).value
        y_pred = extract_predictions(ltn_pred, n)
    acc, prec, rec, f1 = compute_metrics(y_true, y_pred)

    all_results.append({
        "Run": run + 1,
        "Predicate": "Below",
        "satAgg": sat,
        "Accuracy": acc,
        "Precision": prec,
        "Recall": rec,
        "F1": f1
    })

    # ------------------------------
    # IN BETWEEN (ternário)
    # ------------------------------
    X, y_true = build_inbetween_dataset(objects)
    with torch.no_grad():
        ltn_pred = InBetween(x_var, y_var, z_var).value
        y_pred = extract_ternary_predictions(ltn_pred, n)
    acc, prec, rec, f1 = compute_metrics(y_true, y_pred)

    all_results.append({
        "Run": run + 1,
        "Predicate": "InBetween",
        "satAgg": sat,
        "Accuracy": acc,
        "Precision": prec,
        "Recall": rec,
        "F1": f1
    })


 Execução 1
satAgg = 0.9998
satAgg = 0.9998

 Execução 2

 Execução 2
satAgg = 0.9998

 Execução 3
satAgg = 0.9998

 Execução 4
satAgg = 0.9998

 Execução 5
satAgg = 0.9998


## Dataframe para o relatório

In [18]:
df = pd.DataFrame(all_results)
df

,Run,Predicate,satAgg,Accuracy,Precision,Recall,F1
0,1,LeftOf,0.999843,0.500000,0.000000,0.000000,0.000000
1,1,RightOf,0.999843,0.500000,0.000000,0.000000,0.000000
2,1,CloseTo,0.999843,0.093333,0.091973,0.982143,0.168196
3,1,Above,0.999843,0.500000,0.500000,1.000000,0.666667
4,1,Below,0.999843,0.500000,0.000000,0.000000,0.000000
5,1,InBetween,0.999843,0.333333,0.333333,1.000000,0.500000
6,2,LeftOf,0.999843,0.500000,0.000000,0.000000,0.000000
7,2,RightOf,0.999843,0.500000,0.000000,0.000000,0.000000
8,2,CloseTo,0.999843,0.108333,0.107203,0.969697,0.193062
9,2,Above,0.999843,0.500000,0.500000,1.000000,0.666667


In [19]:
df.groupby("Predicate")[["Accuracy", "Precision", "Recall", "F1"]].mean()

,Accuracy,Precision,Recall,F1
Predicate,,,,
Above,0.500000,0.500000,1.000000,0.666667
Below,0.500000,0.000000,0.000000,0.000000
CloseTo,0.111333,0.099110,0.976267,0.179613
InBetween,0.333333,0.333333,1.000000,0.500000
LeftOf,0.500000,0.000000,0.000000,0.000000
RightOf,0.500000,0.000000,0.000000,0.000000
